In [1]:
import pandas as pd
import numpy as np

In [2]:
news_df = pd.read_pickle("cleaned_news.pkl")
news_ids = set(news_df["newsID"])

In [12]:

# Load behaviors data
behaviors_cols = ["impression_id", "user_id", "time", "history", "impressions"]
behaviors_df = pd.read_csv("train_data/behaviors.tsv", sep="\t", header=None, names=behaviors_cols)
display(behaviors_df.head())
display(len(behaviors_df))

,impression_id,user_id,time,history,impressions
0,1,U87243,11/10/2019 11:30:54 AM,N8668 N39081 N65259 N79529 N73408 N43615 N2937...,N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...
1,2,U598644,11/12/2019 1:45:29 PM,N56056 N8726 N70353 N67998 N83823 N111108 N107...,N47996-0 N82719-0 N117066-0 N8491-0 N123784-0 ...
2,3,U532401,11/13/2019 11:23:03 AM,N128643 N87446 N122948 N9375 N82348 N129412 N5...,N103852-0 N53474-0 N127836-0 N47925-1
3,4,U593596,11/12/2019 12:24:09 PM,N31043 N39592 N4104 N8223 N114581 N92747 N1207...,N38902-0 N76434-0 N71593-0 N100073-0 N108736-0...
4,5,U239687,11/14/2019 8:03:01 PM,N65250 N122359 N71723 N53796 N41663 N41484 N11...,N76209-0 N48841-0 N67937-0 N62235-0 N6307-0 N3...


2232748

In [3]:
# Check for duplicates
duplicate_impressions = behaviors_df.duplicated(subset=["impression_id"]).sum()
print(f"Duplicate impression IDs: {duplicate_impressions}")

# Check missing values
print("\nMissing values in behaviors data:")
print(behaviors_df.isnull().sum())

Duplicate impression IDs: 0

Missing values in behaviors data:
impression_id        0
user_id              0
time                 0
history          46065
impressions          0
dtype: int64


In [5]:
# Total number of rows
total_rows = len(behaviors_df)

# Number of missing history values
missing_history_count = behaviors_df['history'].isna().sum()

# Percentage missing
missing_history_pct = (missing_history_count / total_rows) * 100

print(f"Missing history: {missing_history_count} rows ({missing_history_pct:.2f}%)")


Missing history: 46065 rows (2.06%)


In [7]:
missing_history_df = behaviors_df[behaviors_df['history'].isna()]
# Unique users with missing history
unique_users_missing = missing_history_df['user_id'].nunique()
print(f"Unique users with missing history: {unique_users_missing}")

# Click rate (optional — only if you've already exploded impressions)
if 'clicked' in missing_history_df.columns:
    click_rate = missing_history_df['clicked'].mean()
    print(f"Click rate for users with missing history: {click_rate:.2%}")

# Count by timestamp (if timestamp column exists)
if 'time' in missing_history_df.columns:
    print(missing_history_df['time'].value_counts().head())


Unique users with missing history: 12857
time
11/13/2019 12:43:55 PM    4
11/13/2019 12:13:56 PM    4
11/14/2019 9:18:56 AM     4
11/12/2019 5:26:43 AM     4
11/14/2019 1:28:28 PM     4
Name: count, dtype: int64


Even though about 2% of the rows in my behaviors_df dataset have missing history values, I decided to keep them for this stage of the project. Right now, I'm focused on measuring headline click-through rates (CTR) — not building a personalized recommendation model.

Since CTR is calculated at the headline level, it doesn’t really matter whether the user has a reading history or not. In fact, keeping these rows actually helps me include more impressions and clicks, which makes my headline CTR stats more reliable.

Later on, if I shift focus to user behavior or personalization, I might filter or treat these rows differently. But for now, they’re staying in

In [15]:
import pandas as pd

# Only sample read the first 10,000 rows
behaviors_df = pd.read_csv(
    'train_data/behaviors.tsv',
    sep='\t',
    header=None,
    names=['impression_id', 'user_id', 'time', 'history', 'impressions'],
    nrows=10000
)
len(behaviors_df)

10000

The behaviors_df dataset contains around 2 million rows, and each row includes a list of article impressions in the form of strings like "N333-0 N444-1 N555-0". Each item represents an article shown to the user (a news_id) and whether it was clicked (1 for clicked, 0 for not clicked).

When you expand this format so that each impression becomes its own row, the dataset grows significantly—potentially into tens of millions of rows. To avoid overwhelming memory and overloading the machine, I started with a sample of 10,000 rows to test and validate the expansion logic.

The plan is to roll this out in larger batches, likely in increments of 100,000 rows at a time. This way, I can scale the analysis gradually while avoiding performance issues. It allows me to explore headline-level click-through rates (CTR) in a controlled, resource-efficient way.

In [17]:
impressions_expanded = []

for _, row in behaviors_df.iterrows():
    try:
        impressions = row['impressions'].split()
        for item in impressions:
            if '-' in item:
                news_id, clicked = item.split('-')
                impressions_expanded.append({
                    'impression_id': row['impression_id'],
                    'user_id': row['user_id'],
                    'news_id': news_id,
                    'clicked': int(clicked)
                })
    except Exception as e:
        print(f"Skipping row due to error: {e}")
        continue
impressions_expanded

[{'impression_id': 1, 'user_id': 'U87243', 'news_id': 'N78206', 'clicked': 0},
 {'impression_id': 1, 'user_id': 'U87243', 'news_id': 'N26368', 'clicked': 0},
 {'impression_id': 1, 'user_id': 'U87243', 'news_id': 'N7578', 'clicked': 0},
 {'impression_id': 1, 'user_id': 'U87243', 'news_id': 'N58592', 'clicked': 0},
 {'impression_id': 1, 'user_id': 'U87243', 'news_id': 'N19858', 'clicked': 0},
 {'impression_id': 1, 'user_id': 'U87243', 'news_id': 'N58258', 'clicked': 0},
 {'impression_id': 1, 'user_id': 'U87243', 'news_id': 'N18478', 'clicked': 0},
 {'impression_id': 1, 'user_id': 'U87243', 'news_id': 'N2591', 'clicked': 0},
 {'impression_id': 1, 'user_id': 'U87243', 'news_id': 'N97778', 'clicked': 0},
 {'impression_id': 1, 'user_id': 'U87243', 'news_id': 'N32954', 'clicked': 0},
 {'impression_id': 1, 'user_id': 'U87243', 'news_id': 'N94157', 'clicked': 1},
 {'impression_id': 1, 'user_id': 'U87243', 'news_id': 'N39404', 'clicked': 0},
 {'impression_id': 1, 'user_id': 'U87243', 'news_id': 

In the behaviors dataset, each row can contain multiple article impressions in the form of strings like `"N1234-1 N2345-0"`. That means a single row represents a full impression session, not individual article views.

To analyze things like headline click-through rate (CTR), I needed to split or expand those multi-item strings so that each row would represent one article impression. This way, I can calculate how each article performs on its own — which wouldn’t be possible if they were all bundled together in one string.

This means the user saw 3 articles during a session, and clicked only one of them. But since they're all bundled into a single row, it's impossible to calculate individual headline performance.

In the raw `behaviors_df`, each row contains a list of article impressions like`"N333-0 N444-1 N555-0"` but as shown above when expanded these open up to show impressions and click per user and per article and most importantly the click-through rate (CTR) for each headline. This matters because I'm analyzing headline performance — and in this dataset, each `news_id` is tied to a specific headline. So CTR per `news_id` is essentially CTR per headline.

In [18]:
impressions_df = pd.DataFrame(impressions_expanded)
print(f"Expanded to {len(impressions_df)} impression records")

Expanded to 371707 impression records


In [19]:
# Check for invalid news IDs (not in news_df)
invalid_news_ids = impressions_df[~impressions_df["news_id"].isin(news_ids)]
print(f"Impression records with invalid news IDs: {len(invalid_news_ids)}")

Impression records with invalid news IDs: 0


In [20]:
# Check click distribution
clicks = impressions_df["clicked"].sum()
total = len(impressions_df)
print(f"\nOverall CTR: {clicks/total:.4f} ({clicks} clicks out of {total} impressions)")


Overall CTR: 0.0410 (15247 clicks out of 371707 impressions)


This gives us a baseline where 4.1% is our benchmark for evaluating which headlines perform above or below average.

- By headline: which ones score above 4.1%?

- By category: do some consistently outperform others?

- By user segment (returning vs. new): does history matter?

In [21]:
# Check for articles with too few impressions (unreliable CTR)
article_impressions = impressions_df.groupby("news_id").size()
low_impression_articles = (article_impressions < 5).sum()
print(f"Articles with fewer than 5 impressions: {low_impression_articles}")

Articles with fewer than 5 impressions: 5711


In [22]:

# Check for extreme CTRs (potential data issues)
article_ctrs = impressions_df.groupby("news_id")["clicked"].mean()
suspicious_ctrs = ((article_ctrs == 0) | (article_ctrs > 0.7)).sum()
print(f"Articles with suspicious CTRs (0 or >70%): {suspicious_ctrs}")

# Potential adjustments based on exploration:
# 1. Filter out articles with too few impressions (e.g., < 10)
# 2. Investigate articles with extreme CTRs
# 3. Set minimum threshold for impression count when calculating CTR

Articles with suspicious CTRs (0 or >70%): 7079
